In [1]:
# mount driver
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/CardiacSeg/UNETCNX/CHGH

# install dependents
!pip install -q "monai-weekly[nibabel, tqdm, einops]"
!python -c "import matplotlib" || pip install -q matplotlib
!pip install -q timm
!pip install -U -q openmim
!mim install -U -q mmcv-full
!pip install ml-collections
!pip install tensorboardX
%matplotlib inline

# sync python module
%load_ext autoreload
%autoreload 2

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1jz_DGnICBmKWCr_JL904PDQdIEK0_EQG/CardiacSeg/UNETCNX/CHGH
     |████████████████████████████████| 1.2 MB 8.5 MB/s 
     |████████████████████████████████| 41 kB 516 kB/s 
     |████████████████████████████████| 549 kB 37.9 MB/s 
     |████████████████████████████████| 182 kB 75.7 MB/s 
     |████████████████████████████████| 50 kB 6.8 MB/s 
     |████████████████████████████████| 237 kB 64.5 MB/s 
     |████████████████████████████████| 51 kB 5.8 MB/s 
     |████████████████████████████████| 46.0 MB 2.1 MB/s 
     |████████████████████████████████| 190 kB 52.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 77 kB 6.2 MB/s 
  Created wheel for ml-collections: filename=ml_collections-0.1.1-py3-none-any.whl size=94524 sha256=041cdca7e2a963db9521cc9c1384b83594bdb7fbc43d9110152bdcdabe0ec250
  Stored in directory: /root/.cache/pip/w

In [2]:
import sys
# set package path
sys.path.append("/content/drive/MyDrive/CardiacSeg")

import os
from pathlib import PurePath

from monai.transforms import (
    Compose,
    LoadImaged,
)

from data_utils.json_dataset import generate_dataset_json

## Gen data dict json

In [ ]:
# import sys

# # set package path
# sys.path.append("/content/drive/MyDrive/CardiacSeg")

# from data_utils.chgh_dataset import get_data_dicts
# from data_utils.io import save_json

# exp_name = 'exp_2_2'
# model_dir = f'/content/drive/MyDrive/CardiacSeg/SwinUNETR/CHGH/pretrain/{exp_name}'
# root_data_dir = '/content/drive/MyDrive/CardiacSeg/dataset/CHGH'
# data_dir = '/content/drive/MyDrive/CardiacSeg/dataset/CHGH/pretrain'
# data_dicts_json = os.path.join(root_data_dir, 'data_dicts', f'{exp_name}_pretrain.json')
# data_dicts = get_data_dicts(data_dir)
# data_dicts
# tr_data_dicts = data_dicts[0:3] + data_dicts[4:7] + data_dicts[8:-1]
# tt_data_dicts = [data_dicts[3], data_dicts[7], data_dicts[-1]]
# target_data_dicts = tr_data_dicts + tt_data_dicts + tt_data_dicts
# save_json(target_data_dicts, data_dicts_json)
# target_data_dicts

save json to /content/drive/MyDrive/CardiacSeg/dataset/CHGH/data_dicts/exp_2_2_pretrain.json


[{'image': '/content/drive/MyDrive/CardiacSeg/dataset/CHGH/pretrain/pid_05/pid_05.nii.gz',
  'label': '/content/drive/MyDrive/CardiacSeg/dataset/CHGH/pretrain/pid_05/pid_05_gt.nii.gz'},
 {'image': '/content/drive/MyDrive/CardiacSeg/dataset/CHGH/pretrain/pid_06/pid_06.nii.gz',
  'label': '/content/drive/MyDrive/CardiacSeg/dataset/CHGH/pretrain/pid_06/pid_06_gt.nii.gz'},
 {'image': '/content/drive/MyDrive/CardiacSeg/dataset/CHGH/pretrain/pid_07/pid_07.nii.gz',
  'label': '/content/drive/MyDrive/CardiacSeg/dataset/CHGH/pretrain/pid_07/pid_07_gt.nii.gz'},
 {'image': '/content/drive/MyDrive/CardiacSeg/dataset/CHGH/pretrain/pid_21/pid_21.nii.gz',
  'label': '/content/drive/MyDrive/CardiacSeg/dataset/CHGH/pretrain/pid_21/pid_21_gt.nii.gz'},
 {'image': '/content/drive/MyDrive/CardiacSeg/dataset/CHGH/pretrain/pid_23/pid_23.nii.gz',
  'label': '/content/drive/MyDrive/CardiacSeg/dataset/CHGH/pretrain/pid_23/pid_23_gt.nii.gz'},
 {'image': '/content/drive/MyDrive/CardiacSeg/dataset/CHGH/pretrain/pi

## Generate data json file

In [ ]:
# data_json_pth = os.path.join(root_data_dir, 'data_dicts', f'{exp_name}_pretrain_data.json')
# generate_dataset_json(
#     target_data_dicts,
#     data_json_pth,
#     dataset_name='segthor',
#     labels={0: 'background', 2: 'heart'},
#     tensorImageSize='3D',
#     sort_keys=True,
#     modalities=['CT'],
#     license='see challenge website',
#     split_train_ratio=0.8,
#     fold=3,
#     num_fold=4,
# )

fold: 3
train files (11): ['pid_05', 'pid_06', 'pid_07', 'pid_21', 'pid_23', 'pid_24', 'pid_30', 'pid_34', 'pid_57', 'pid_110', 'pid_1002']
val files (3): ['pid_08', 'pid_27', 'pid_1003']
test files (3): ['pid_08', 'pid_27', 'pid_1003']
save json to /content/drive/MyDrive/CardiacSeg/dataset/CHGH/data_dicts/exp_2_2_pretrain_data.json


## Pretrain

In [3]:
exp_name = 'exp_2_2_x0'
json_file_name = 'exp_2_2_pretrain_data.json'
model_name = 'unetcnx_x0'

root_dir = '/content/drive/MyDrive/CardiacSeg/UNETCNX/CHGH'
root_data_dir = '/content/drive/MyDrive/CardiacSeg/dataset/CHGH'

data_dir = os.path.join(root_data_dir, 'dataset_2')
model_dir = os.path.join(root_dir, 'models', 'pretrain', exp_name)
log_dir = os.path.join(root_dir, 'logs', 'pretrain', exp_name)


data_dicts_json = os.path.join(root_data_dir, 'data_dicts', json_file_name)
best_model_path = os.path.join(model_dir, 'model_bestValRMSE.pt')
final_model_path = os.path.join(model_dir, 'model_final.pt')

In [4]:
!python /content/drive/MyDrive/CardiacSeg/expers/pretrain/train.py\
--use_checkpoint \
--batch_size=1 \
--num_steps=100000 \
--lrdecay \
--eval_num=100 \
--model_name={model_name} \
--logdir={log_dir} \
--model_dir={model_dir} \
--data_name={data_dicts_json} \
--space_x=0.7 \
--space_y=0.7 \
--space_z=1.0 \
--resume={final_model_path}
# --lr=<Lr> 
# --roi_x=<Roi_x> \
# --roi_y=<Roi_y> \
# --roi_z=<Roi_z> \

2022-12-14 02:15:53,102 - Generating grammar tables from /usr/lib/python3.8/lib2to3/Grammar.txt
2022-12-14 02:15:53,120 - Generating grammar tables from /usr/lib/python3.8/lib2to3/PatternGrammar.txt
/usr/local/lib/python3.8/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
cuda is available
Training with a single process on 1 GPUs.
model: unetcnx_x0
use warmup cosine schedule
load model: /content/drive/MyDrive/CardiacSeg/UNETCNX/CHGH/models/pretrain/exp_2_2_x0/model_final.pt
['/content/drive/MyDrive/CardiacSeg/dataset/CHGH/data_dicts/exp_2_2_pretrain_data.json']
load json from /content/drive/MyDrive/CardiacSeg/dataset/CHGH/data_dicts/exp_2_2_pre